In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import datetime as dt
from bs4 import BeautifulSoup

tqdm.pandas()

In [2]:
# Get cities and contextuals

url = 'http://www.globalcovenantofmayors.org/wp-json/map/v1/cities'
cities = requests.get(url).json()
cities = pd.DataFrame(cities)
cities.to_csv('global_covenant_cities.csv')

In [3]:
# Some missing data now - need to remove those
cities[cities['URL'].isnull()]
cities = cities.dropna(how = 'all')

In [4]:
def get_page(url): 
    page = BeautifulSoup(requests.get(url).content, 'lxml')
    return page

cities['page'] = cities['URL'].progress_apply(get_page)

100%|██████████| 7497/7497 [47:33<00:00,  3.43it/s]  


In [5]:
# phase in [Commitment, Inventory, Target, Plan]
def phase(phase, page): 
    p = page.find_all('img', {'class': 'phase_badge'})
    for i in p: 
        if i['alt'] == phase:
            return True
    return False

In [6]:
cities['commitment_phase'] = cities['page'].progress_apply(lambda x: phase('Commitment', x))
cities['inventory_phase'] = cities['page'].progress_apply(lambda x: phase('Inventory', x))
cities['target_phase'] = cities['page'].progress_apply(lambda x: phase('Target', x))
cities['plan_phase'] = cities['page'].progress_apply(lambda x: phase('Plan', x))
# are there more?


100%|██████████| 7497/7497 [00:23<00:00, 313.66it/s]


In [12]:
# text in [Total Annual GHG Emissions, Targets:, Potential Impact of GHG Emissions Target]
def commitment_data(text, page): 
    p = page.find('div', {'class': 'all_climate_data'}).find_all('div', {'class': 'vc_row'})
    for i in p:
        label = i.find('div', {'class': 'label'})
        if label:
            if text in label.text:
                value = i.find('span', {'class': 'counter_number'})
                if value and '...' not in value.text: 
                    value = value.text
                    value = value.replace(',', '').replace('%', '').replace('\t', '')
                    value = float(value) 
                    return value
          
def target_year(page): 
    p = page.find('div', {'class': 'all_climate_data'}).find_all('div', {'class': 'vc_row'})
    for i in p:
        label = i.find('div', {'class': 'label'})
        if label:
            if 'Targets:' in label.text:
                value = i.find('p')
                if value.text: 
                    value = value.text.split('by ')[1]
                    value = int(value)
                    return value

In [13]:
cities['total_ghg_emissions'] = cities['page'].progress_apply(lambda x: commitment_data('Total Annual GHG Emissions', x))
cities['percent_reduction'] = cities['page'].progress_apply(lambda x: commitment_data('Targets:', x))
cities['potential_impact'] = cities['page'].progress_apply(lambda x: commitment_data('Potential Impact', x))
cities['target_year'] = cities['page'].progress_apply(lambda x: target_year(x))




  0%|          | 0/7497 [00:00<?, ?it/s]


  0%|          | 13/7497 [00:00<00:58, 127.79it/s]


  1%|          | 38/7497 [00:00<00:50, 149.02it/s]


  1%|          | 69/7497 [00:00<00:42, 176.30it/s]


  1%|▏         | 100/7497 [00:00<00:36, 201.42it/s]


  2%|▏         | 134/7497 [00:00<00:32, 228.76it/s]


  2%|▏         | 167/7497 [00:00<00:29, 250.40it/s]


  3%|▎         | 198/7497 [00:00<00:27, 264.27it/s]


  3%|▎         | 228/7497 [00:00<00:26, 272.78it/s]


  3%|▎         | 257/7497 [00:00<00:26, 272.81it/s]


  4%|▍         | 286/7497 [00:01<00:26, 276.64it/s]


  4%|▍         | 318/7497 [00:01<00:25, 286.24it/s]


  5%|▍         | 351/7497 [00:01<00:24, 297.54it/s]


  5%|▌         | 382/7497 [00:01<00:27, 260.50it/s]


  5%|▌         | 410/7497 [00:01<00:27, 262.42it/s]


  6%|▌         | 443/7497 [00:01<00:25, 279.08it/s]


  6%|▋         | 473/7497 [00:01<00:24, 284.84it/s]


  7%|▋         | 504/7497 [00:01<00:24, 288.95it/s]


  7%|▋         | 535/7497 [00:01<00:23,

 58%|█████▊    | 4323/7497 [00:15<00:11, 276.09it/s]


 58%|█████▊    | 4352/7497 [00:16<00:11, 279.53it/s]


 58%|█████▊    | 4380/7497 [00:16<00:11, 279.04it/s]


 59%|█████▉    | 4410/7497 [00:16<00:10, 283.64it/s]


 59%|█████▉    | 4439/7497 [00:16<00:11, 266.86it/s]


 60%|█████▉    | 4466/7497 [00:16<00:11, 255.88it/s]


 60%|█████▉    | 4492/7497 [00:16<00:12, 247.07it/s]


 60%|██████    | 4519/7497 [00:16<00:11, 253.01it/s]


 61%|██████    | 4548/7497 [00:16<00:11, 262.52it/s]


 61%|██████    | 4576/7497 [00:16<00:10, 266.21it/s]


 61%|██████▏   | 4603/7497 [00:17<00:11, 260.60it/s]


 62%|██████▏   | 4631/7497 [00:17<00:10, 263.33it/s]


 62%|██████▏   | 4658/7497 [00:17<00:11, 257.90it/s]


 63%|██████▎   | 4689/7497 [00:17<00:10, 270.31it/s]


 63%|██████▎   | 4718/7497 [00:17<00:10, 273.07it/s]


 63%|██████▎   | 4746/7497 [00:17<00:10, 265.39it/s]


 64%|██████▎   | 4773/7497 [00:17<00:10, 263.89it/s]


 64%|██████▍   | 4800/7497 [00:17<00:10, 256.12it/s]


 64%|█████

 14%|█▍        | 1064/7497 [00:03<00:21, 300.46it/s]


 15%|█▍        | 1095/7497 [00:03<00:21, 291.55it/s]


 15%|█▌        | 1125/7497 [00:03<00:22, 281.04it/s]


 15%|█▌        | 1157/7497 [00:03<00:21, 289.82it/s]


 16%|█▌        | 1188/7497 [00:04<00:21, 293.64it/s]


 16%|█▋        | 1220/7497 [00:04<00:20, 300.46it/s]


 17%|█▋        | 1251/7497 [00:04<00:20, 301.66it/s]


 17%|█▋        | 1282/7497 [00:04<00:20, 296.51it/s]


 18%|█▊        | 1314/7497 [00:04<00:20, 300.82it/s]


 18%|█▊        | 1345/7497 [00:04<00:20, 302.91it/s]


 18%|█▊        | 1376/7497 [00:04<00:21, 288.22it/s]


 19%|█▉        | 1408/7497 [00:04<00:20, 294.77it/s]


 19%|█▉        | 1441/7497 [00:04<00:20, 301.41it/s]


 20%|█▉        | 1472/7497 [00:05<00:20, 296.35it/s]


 20%|██        | 1503/7497 [00:05<00:20, 299.63it/s]


 20%|██        | 1535/7497 [00:05<00:19, 303.84it/s]


 21%|██        | 1566/7497 [00:05<00:19, 301.30it/s]


 21%|██▏       | 1597/7497 [00:05<00:20, 289.58it/s]


 22%|██▏  

 77%|███████▋  | 5776/7497 [00:19<00:05, 305.39it/s]


 77%|███████▋  | 5809/7497 [00:19<00:05, 310.86it/s]


 78%|███████▊  | 5841/7497 [00:19<00:05, 298.37it/s]


 78%|███████▊  | 5872/7497 [00:19<00:05, 297.04it/s]


 79%|███████▉  | 5905/7497 [00:20<00:05, 305.91it/s]


 79%|███████▉  | 5937/7497 [00:20<00:05, 309.73it/s]


 80%|███████▉  | 5969/7497 [00:20<00:05, 298.99it/s]


 80%|████████  | 6001/7497 [00:20<00:04, 302.63it/s]


 80%|████████  | 6034/7497 [00:20<00:04, 308.84it/s]


 81%|████████  | 6068/7497 [00:20<00:04, 316.05it/s]


 81%|████████▏ | 6102/7497 [00:20<00:04, 320.39it/s]


 82%|████████▏ | 6135/7497 [00:20<00:04, 317.86it/s]


 82%|████████▏ | 6167/7497 [00:20<00:04, 305.13it/s]


 83%|████████▎ | 6201/7497 [00:20<00:04, 311.58it/s]


 83%|████████▎ | 6234/7497 [00:21<00:03, 316.22it/s]


 84%|████████▎ | 6268/7497 [00:21<00:03, 320.47it/s]


 84%|████████▍ | 6301/7497 [00:21<00:03, 317.92it/s]


 84%|████████▍ | 6334/7497 [00:21<00:03, 319.83it/s]


 85%|█████

 31%|███       | 2309/7497 [00:10<00:26, 194.05it/s]


 31%|███       | 2330/7497 [00:10<00:26, 193.25it/s]


 31%|███▏      | 2355/7497 [00:10<00:24, 207.00it/s]


 32%|███▏      | 2378/7497 [00:10<00:24, 211.77it/s]


 32%|███▏      | 2400/7497 [00:10<00:24, 204.22it/s]


 32%|███▏      | 2421/7497 [00:10<00:25, 199.66it/s]


 33%|███▎      | 2442/7497 [00:11<00:26, 194.36it/s]


 33%|███▎      | 2462/7497 [00:11<00:27, 185.81it/s]


 33%|███▎      | 2484/7497 [00:11<00:25, 193.50it/s]


 33%|███▎      | 2506/7497 [00:11<00:25, 198.19it/s]


 34%|███▎      | 2527/7497 [00:11<00:26, 185.75it/s]


 34%|███▍      | 2546/7497 [00:11<00:30, 161.46it/s]


 34%|███▍      | 2563/7497 [00:11<00:34, 142.64it/s]


 35%|███▍      | 2587/7497 [00:11<00:30, 162.15it/s]


 35%|███▍      | 2611/7497 [00:11<00:27, 177.40it/s]


 35%|███▌      | 2631/7497 [00:12<00:28, 169.76it/s]


 35%|███▌      | 2650/7497 [00:12<00:28, 169.40it/s]


 36%|███▌      | 2668/7497 [00:12<00:31, 155.22it/s]


 36%|███▌ 

 86%|████████▌ | 6443/7497 [00:26<00:03, 279.53it/s]


 86%|████████▋ | 6472/7497 [00:26<00:03, 276.37it/s]


 87%|████████▋ | 6500/7497 [00:26<00:03, 255.70it/s]


 87%|████████▋ | 6529/7497 [00:26<00:03, 264.79it/s]


 87%|████████▋ | 6559/7497 [00:26<00:03, 271.72it/s]


 88%|████████▊ | 6587/7497 [00:26<00:03, 270.38it/s]


 88%|████████▊ | 6620/7497 [00:27<00:03, 284.06it/s]


 89%|████████▊ | 6649/7497 [00:27<00:03, 273.91it/s]


 89%|████████▉ | 6677/7497 [00:27<00:03, 268.02it/s]


 89%|████████▉ | 6709/7497 [00:27<00:02, 279.75it/s]


 90%|████████▉ | 6740/7497 [00:27<00:02, 287.58it/s]


 90%|█████████ | 6772/7497 [00:27<00:02, 293.60it/s]


 91%|█████████ | 6803/7497 [00:27<00:02, 295.98it/s]


 91%|█████████ | 6833/7497 [00:27<00:02, 290.55it/s]


 92%|█████████▏| 6864/7497 [00:27<00:02, 294.62it/s]


 92%|█████████▏| 6894/7497 [00:27<00:02, 292.99it/s]


 92%|█████████▏| 6926/7497 [00:28<00:01, 298.61it/s]


 93%|█████████▎| 6956/7497 [00:28<00:02, 239.27it/s]


 93%|█████

 46%|████▌     | 3424/7497 [00:11<00:14, 273.97it/s]


 46%|████▌     | 3452/7497 [00:11<00:14, 270.33it/s]


 46%|████▋     | 3480/7497 [00:11<00:14, 269.44it/s]


 47%|████▋     | 3508/7497 [00:12<00:14, 271.15it/s]


 47%|████▋     | 3536/7497 [00:12<00:14, 272.37it/s]


 48%|████▊     | 3564/7497 [00:12<00:14, 273.22it/s]


 48%|████▊     | 3592/7497 [00:12<00:16, 243.22it/s]


 48%|████▊     | 3617/7497 [00:12<00:16, 239.07it/s]


 49%|████▊     | 3642/7497 [00:12<00:16, 234.30it/s]


 49%|████▉     | 3666/7497 [00:12<00:16, 233.42it/s]


 49%|████▉     | 3690/7497 [00:12<00:17, 211.87it/s]


 50%|████▉     | 3712/7497 [00:12<00:17, 213.82it/s]


 50%|████▉     | 3734/7497 [00:13<00:17, 215.15it/s]


 50%|█████     | 3756/7497 [00:13<00:19, 188.44it/s]


 50%|█████     | 3776/7497 [00:13<00:27, 133.88it/s]


 51%|█████     | 3793/7497 [00:13<00:30, 120.36it/s]


 51%|█████     | 3815/7497 [00:13<00:26, 138.81it/s]


 51%|█████     | 3840/7497 [00:13<00:22, 159.05it/s]


 51%|█████

 97%|█████████▋| 7267/7497 [00:28<00:00, 244.16it/s]


 97%|█████████▋| 7295/7497 [00:28<00:00, 253.36it/s]


 98%|█████████▊| 7321/7497 [00:28<00:00, 246.47it/s]


 98%|█████████▊| 7349/7497 [00:28<00:00, 253.73it/s]


 98%|█████████▊| 7375/7497 [00:28<00:00, 255.03it/s]


 99%|█████████▊| 7403/7497 [00:28<00:00, 260.05it/s]


 99%|█████████▉| 7430/7497 [00:28<00:00, 251.43it/s]


 99%|█████████▉| 7457/7497 [00:29<00:00, 256.20it/s]


100%|█████████▉| 7486/7497 [00:29<00:00, 262.79it/s]


100%|██████████| 7497/7497 [00:29<00:00, 257.02it/s]

In [14]:
# text in [Action, Adaptation]
def plan_url(text, page):
    p = page.find('div', {'class': 'all_climate_data'})
    if p is None:
        return None
    p = p.find_all('div', {'class': 'vc_row'})
    for i in p:
        links = i.find_all('a')
        for l in links:
            if text in l.text:
                url = l['href']
                return url

In [15]:
cities.loc[3503, 'page'].find('div', {'class': 'all_climate_data'}).find_all('div', {'class': 'vc_row'})

[<div class="vc_row wpb_row vc_inner row-fluid">
 <div class="cell label has_tooltip">Total Annual GHG Emissions: <i class="fa fa-question-circle">
 <div class="tooltip">Total amount of greenhouse gas generated within a city’s boundary per year and released into the atmosphere, expressed as CO2 equivalent.</div> </i>
 </div>
 <div class="cell faded"><span class="counter_number"><strong>...</strong><sup>*</sup></span>
 <p></p>
 <p class="faded fc"><em><sup>*</sup> Data forthcoming</em></p> </div>
 </div>, <div class="vc_row wpb_row vc_inner row-fluid">
 <div class="vc_col-sm-6">
 <p class="regular">Buildings</p>
 <div class="d3" data-label="" data-value="" id="div1"></div>
 <p class="faded fc"><em><sup>*</sup> Data forthcoming</em></p> </div>
 <div class="vc_col-sm-6">
 <p class="regular">Transportation</p>
 <div class="d3" data-label="" data-value="" id="div2"></div>
 <p class="faded fc"><em><sup>*</sup> Data forthcoming</em></p> </div>
 </div>, <div class="vc_row wpb_row vc_inner ro

In [16]:
cities['action_plan_url'] = cities['page'].progress_apply(lambda x: plan_url('Action', x))
cities['adaptation_plan_url'] = cities['page'].progress_apply(lambda x: plan_url('Adaptation', x))
# others?




  0%|          | 0/7497 [00:00<?, ?it/s]


  0%|          | 33/7497 [00:00<00:22, 327.21it/s]


  1%|          | 69/7497 [00:00<00:22, 335.62it/s]


  1%|▏         | 105/7497 [00:00<00:21, 341.85it/s]


  2%|▏         | 140/7497 [00:00<00:21, 343.48it/s]


  2%|▏         | 178/7497 [00:00<00:20, 351.96it/s]


  3%|▎         | 217/7497 [00:00<00:20, 360.89it/s]


  3%|▎         | 255/7497 [00:00<00:19, 366.04it/s]


  4%|▍         | 293/7497 [00:00<00:19, 368.26it/s]


  4%|▍         | 330/7497 [00:00<00:19, 367.95it/s]


  5%|▍         | 368/7497 [00:01<00:19, 368.57it/s]


  5%|▌         | 405/7497 [00:01<00:19, 366.33it/s]


  6%|▌         | 444/7497 [00:01<00:18, 372.39it/s]


  6%|▋         | 481/7497 [00:01<00:19, 356.23it/s]


  7%|▋         | 520/7497 [00:01<00:19, 363.85it/s]


  7%|▋         | 559/7497 [00:01<00:18, 369.86it/s]


  8%|▊         | 596/7497 [00:01<00:18, 369.58it/s]


  8%|▊         | 635/7497 [00:01<00:18, 373.59it/s]


  9%|▉         | 674/7497 [00:01<00:18

 70%|███████   | 5252/7497 [00:16<00:06, 330.35it/s]


 71%|███████   | 5287/7497 [00:16<00:09, 235.90it/s]


 71%|███████   | 5316/7497 [00:16<00:09, 235.48it/s]


 71%|███████▏  | 5348/7497 [00:16<00:08, 254.66it/s]


 72%|███████▏  | 5378/7497 [00:16<00:07, 266.25it/s]


 72%|███████▏  | 5415/7497 [00:16<00:07, 290.19it/s]


 73%|███████▎  | 5454/7497 [00:16<00:06, 313.01it/s]


 73%|███████▎  | 5493/7497 [00:17<00:06, 331.25it/s]


 74%|███████▍  | 5531/7497 [00:17<00:05, 342.95it/s]


 74%|███████▍  | 5570/7497 [00:17<00:05, 354.10it/s]


 75%|███████▍  | 5607/7497 [00:17<00:05, 357.98it/s]


 75%|███████▌  | 5644/7497 [00:17<00:05, 354.57it/s]


 76%|███████▌  | 5680/7497 [00:17<00:05, 337.47it/s]


 76%|███████▌  | 5715/7497 [00:17<00:05, 337.40it/s]


 77%|███████▋  | 5754/7497 [00:17<00:04, 349.07it/s]


 77%|███████▋  | 5791/7497 [00:17<00:04, 353.67it/s]


 78%|███████▊  | 5830/7497 [00:18<00:04, 361.08it/s]


 78%|███████▊  | 5867/7497 [00:18<00:04, 360.82it/s]


 79%|█████

 42%|████▏     | 3158/7497 [00:09<00:13, 324.30it/s]


 43%|████▎     | 3191/7497 [00:09<00:13, 324.75it/s]


 43%|████▎     | 3224/7497 [00:09<00:13, 321.77it/s]


 43%|████▎     | 3257/7497 [00:09<00:13, 323.49it/s]


 44%|████▍     | 3290/7497 [00:09<00:13, 318.17it/s]


 44%|████▍     | 3324/7497 [00:09<00:12, 321.85it/s]


 45%|████▍     | 3357/7497 [00:09<00:12, 321.71it/s]


 45%|████▌     | 3390/7497 [00:09<00:12, 319.14it/s]


 46%|████▌     | 3423/7497 [00:09<00:12, 319.72it/s]


 46%|████▌     | 3455/7497 [00:10<00:12, 319.12it/s]


 47%|████▋     | 3487/7497 [00:10<00:13, 300.79it/s]


 47%|████▋     | 3518/7497 [00:10<00:13, 292.55it/s]


 47%|████▋     | 3549/7497 [00:10<00:13, 296.89it/s]


 48%|████▊     | 3582/7497 [00:10<00:12, 303.81it/s]


 48%|████▊     | 3616/7497 [00:10<00:12, 312.32it/s]


 49%|████▊     | 3650/7497 [00:10<00:12, 319.48it/s]


 49%|████▉     | 3683/7497 [00:10<00:11, 318.03it/s]


 50%|████▉     | 3715/7497 [00:10<00:11, 317.91it/s]


 50%|█████

In [17]:
def sector_emissions(sector, page): # sector in [Buildings, Transportation, Waste, Industry, Other]
    p = page.find('div', {'class': 'emissions'})
    if p is None:
        return None
    for i in p.find_all('div', {'class': 'vc_col-sm-6'}):
        # find sector
        s = i.find('p', {'class': 'regular'})
        if s: 
            s = s.text
            
        if s == sector:
            value = i.find('div', {'class': 'd3'})
            if value:
                value = value['data-value']
                if value != '':
                    try:
                        value = float(value)
                        return value
                    except:
                        pass

In [18]:
cities['building_sector_emissions'] = cities['page'].progress_apply(lambda x: sector_emissions('Buildings', x))
cities['transportation_sector_emissions'] = cities['page'].progress_apply(lambda x: sector_emissions('Transportation', x))
cities['waste_sector_emissions'] = cities['page'].progress_apply(lambda x: sector_emissions('Waste', x))
cities['industrial_sector_emissions'] = cities['page'].progress_apply(lambda x: sector_emissions('Industry', x))
cities['other_sector_emissions'] = cities['page'].progress_apply(lambda x: sector_emissions('Other', x))





  0%|          | 0/7497 [00:00<?, ?it/s]


  0%|          | 35/7497 [00:00<00:21, 346.70it/s]


  1%|          | 70/7497 [00:00<00:21, 347.66it/s]


  1%|▏         | 107/7497 [00:00<00:21, 351.43it/s]


  2%|▏         | 144/7497 [00:00<00:20, 355.39it/s]


  2%|▏         | 181/7497 [00:00<00:20, 357.57it/s]


  3%|▎         | 217/7497 [00:00<00:20, 356.90it/s]


  3%|▎         | 252/7497 [00:00<00:20, 353.93it/s]


  4%|▍         | 288/7497 [00:00<00:20, 354.62it/s]


  4%|▍         | 324/7497 [00:00<00:20, 354.21it/s]


  5%|▍         | 359/7497 [00:01<00:20, 350.99it/s]


  5%|▌         | 394/7497 [00:01<00:20, 345.21it/s]


  6%|▌         | 430/7497 [00:01<00:20, 347.39it/s]


  6%|▌         | 467/7497 [00:01<00:19, 352.03it/s]


  7%|▋         | 503/7497 [00:01<00:19, 353.91it/s]


  7%|▋         | 539/7497 [00:01<00:20, 347.10it/s]


  8%|▊         | 574/7497 [00:01<00:19, 347.21it/s]


  8%|▊         | 609/7497 [00:01<00:20, 344.05it/s]


  9%|▊         | 644/7497 [00:01<00:20

 56%|█████▌    | 4192/7497 [00:16<00:12, 270.25it/s]


 56%|█████▋    | 4225/7497 [00:16<00:11, 283.65it/s]


 57%|█████▋    | 4255/7497 [00:16<00:11, 285.52it/s]


 57%|█████▋    | 4284/7497 [00:16<00:11, 277.62it/s]


 58%|█████▊    | 4313/7497 [00:16<00:12, 258.19it/s]


 58%|█████▊    | 4340/7497 [00:17<00:12, 257.34it/s]


 58%|█████▊    | 4367/7497 [00:17<00:12, 254.58it/s]


 59%|█████▊    | 4398/7497 [00:17<00:11, 267.81it/s]


 59%|█████▉    | 4430/7497 [00:17<00:10, 279.85it/s]


 60%|█████▉    | 4462/7497 [00:17<00:10, 289.38it/s]


 60%|█████▉    | 4492/7497 [00:17<00:10, 285.22it/s]


 60%|██████    | 4523/7497 [00:17<00:10, 291.63it/s]


 61%|██████    | 4553/7497 [00:17<00:10, 274.95it/s]


 61%|██████    | 4581/7497 [00:17<00:11, 249.38it/s]


 61%|██████▏   | 4607/7497 [00:17<00:11, 248.65it/s]


 62%|██████▏   | 4633/7497 [00:18<00:11, 245.04it/s]


 62%|██████▏   | 4660/7497 [00:18<00:11, 250.81it/s]


 63%|██████▎   | 4689/7497 [00:18<00:10, 260.89it/s]


 63%|█████

 15%|█▌        | 1130/7497 [00:03<00:27, 230.05it/s]


 15%|█▌        | 1154/7497 [00:04<00:31, 199.50it/s]


 16%|█▌        | 1179/7497 [00:04<00:29, 210.89it/s]


 16%|█▌        | 1211/7497 [00:04<00:26, 234.04it/s]


 17%|█▋        | 1243/7497 [00:04<00:24, 253.48it/s]


 17%|█▋        | 1275/7497 [00:04<00:23, 269.19it/s]


 17%|█▋        | 1307/7497 [00:04<00:21, 282.05it/s]


 18%|█▊        | 1339/7497 [00:04<00:21, 291.04it/s]


 18%|█▊        | 1371/7497 [00:04<00:20, 298.05it/s]


 19%|█▉        | 1406/7497 [00:04<00:19, 311.30it/s]


 19%|█▉        | 1441/7497 [00:05<00:18, 321.33it/s]


 20%|█▉        | 1475/7497 [00:05<00:18, 324.06it/s]


 20%|██        | 1508/7497 [00:05<00:18, 318.50it/s]


 21%|██        | 1541/7497 [00:05<00:18, 316.02it/s]


 21%|██        | 1578/7497 [00:05<00:18, 327.95it/s]


 22%|██▏       | 1612/7497 [00:05<00:17, 328.86it/s]


 22%|██▏       | 1647/7497 [00:05<00:17, 334.22it/s]


 22%|██▏       | 1683/7497 [00:05<00:17, 339.90it/s]


 23%|██▎  

 83%|████████▎ | 6207/7497 [00:19<00:03, 334.88it/s]


 83%|████████▎ | 6241/7497 [00:19<00:03, 326.03it/s]


 84%|████████▎ | 6276/7497 [00:19<00:03, 331.20it/s]


 84%|████████▍ | 6311/7497 [00:19<00:03, 334.47it/s]


 85%|████████▍ | 6345/7497 [00:20<00:04, 246.59it/s]


 85%|████████▌ | 6374/7497 [00:20<00:04, 244.67it/s]


 85%|████████▌ | 6408/7497 [00:20<00:04, 266.66it/s]


 86%|████████▌ | 6444/7497 [00:20<00:03, 287.25it/s]


 86%|████████▋ | 6479/7497 [00:20<00:03, 303.02it/s]


 87%|████████▋ | 6514/7497 [00:20<00:03, 313.68it/s]


 87%|████████▋ | 6550/7497 [00:20<00:02, 325.02it/s]


 88%|████████▊ | 6586/7497 [00:20<00:02, 332.24it/s]


 88%|████████▊ | 6623/7497 [00:21<00:02, 340.16it/s]


 89%|████████▉ | 6658/7497 [00:21<00:02, 339.35it/s]


 89%|████████▉ | 6693/7497 [00:21<00:02, 335.94it/s]


 90%|████████▉ | 6727/7497 [00:21<00:02, 316.23it/s]


 90%|█████████ | 6764/7497 [00:21<00:02, 329.07it/s]


 91%|█████████ | 6800/7497 [00:21<00:02, 337.08it/s]


 91%|█████

 52%|█████▏    | 3865/7497 [00:11<00:11, 322.92it/s]


 52%|█████▏    | 3900/7497 [00:11<00:10, 328.53it/s]


 52%|█████▏    | 3935/7497 [00:11<00:10, 333.03it/s]


 53%|█████▎    | 3971/7497 [00:11<00:10, 339.01it/s]


 53%|█████▎    | 4006/7497 [00:11<00:10, 341.56it/s]


 54%|█████▍    | 4042/7497 [00:12<00:09, 346.16it/s]


 54%|█████▍    | 4078/7497 [00:12<00:09, 347.38it/s]


 55%|█████▍    | 4114/7497 [00:12<00:09, 350.32it/s]


 55%|█████▌    | 4150/7497 [00:12<00:09, 351.38it/s]


 56%|█████▌    | 4186/7497 [00:12<00:09, 349.08it/s]


 56%|█████▋    | 4221/7497 [00:12<00:09, 346.46it/s]


 57%|█████▋    | 4257/7497 [00:12<00:09, 348.64it/s]


 57%|█████▋    | 4292/7497 [00:12<00:09, 346.24it/s]


 58%|█████▊    | 4328/7497 [00:12<00:09, 348.42it/s]


 58%|█████▊    | 4364/7497 [00:12<00:08, 349.42it/s]


 59%|█████▊    | 4401/7497 [00:13<00:08, 353.97it/s]


 59%|█████▉    | 4438/7497 [00:13<00:08, 355.83it/s]


 60%|█████▉    | 4474/7497 [00:13<00:08, 350.08it/s]


 60%|█████

 21%|██        | 1543/7497 [00:04<00:17, 337.54it/s]


 21%|██        | 1577/7497 [00:04<00:17, 330.61it/s]


 22%|██▏       | 1612/7497 [00:04<00:17, 333.88it/s]


 22%|██▏       | 1648/7497 [00:04<00:17, 339.20it/s]


 22%|██▏       | 1682/7497 [00:05<00:17, 338.13it/s]


 23%|██▎       | 1716/7497 [00:05<00:17, 333.89it/s]


 23%|██▎       | 1752/7497 [00:05<00:16, 339.59it/s]


 24%|██▍       | 1787/7497 [00:05<00:16, 340.77it/s]


 24%|██▍       | 1822/7497 [00:05<00:16, 339.75it/s]


 25%|██▍       | 1857/7497 [00:05<00:16, 340.04it/s]


 25%|██▌       | 1893/7497 [00:05<00:16, 345.07it/s]


 26%|██▌       | 1928/7497 [00:05<00:16, 343.75it/s]


 26%|██▌       | 1964/7497 [00:05<00:16, 345.75it/s]


 27%|██▋       | 2000/7497 [00:05<00:15, 347.15it/s]


 27%|██▋       | 2036/7497 [00:06<00:15, 349.14it/s]


 28%|██▊       | 2071/7497 [00:06<00:15, 345.39it/s]


 28%|██▊       | 2106/7497 [00:06<00:18, 284.54it/s]


 29%|██▊       | 2138/7497 [00:06<00:18, 292.90it/s]


 29%|██▉  

 88%|████████▊ | 6613/7497 [00:20<00:02, 333.61it/s]


 89%|████████▊ | 6647/7497 [00:20<00:02, 325.81it/s]


 89%|████████▉ | 6681/7497 [00:20<00:02, 328.27it/s]


 90%|████████▉ | 6717/7497 [00:20<00:02, 334.64it/s]


 90%|█████████ | 6753/7497 [00:20<00:02, 340.90it/s]


 91%|█████████ | 6788/7497 [00:20<00:02, 339.84it/s]


 91%|█████████ | 6823/7497 [00:20<00:02, 329.61it/s]


 91%|█████████▏| 6857/7497 [00:20<00:02, 316.25it/s]


 92%|█████████▏| 6889/7497 [00:21<00:01, 311.15it/s]


 92%|█████████▏| 6921/7497 [00:21<00:02, 286.64it/s]


 93%|█████████▎| 6951/7497 [00:21<00:01, 274.81it/s]


 93%|█████████▎| 6983/7497 [00:21<00:01, 286.73it/s]


 94%|█████████▎| 7013/7497 [00:21<00:01, 289.09it/s]


 94%|█████████▍| 7048/7497 [00:21<00:01, 302.86it/s]


 94%|█████████▍| 7082/7497 [00:21<00:01, 312.80it/s]


 95%|█████████▍| 7116/7497 [00:21<00:01, 319.27it/s]


 95%|█████████▌| 7149/7497 [00:21<00:01, 315.64it/s]


 96%|█████████▌| 7182/7497 [00:22<00:00, 318.17it/s]


 96%|█████

 51%|█████     | 3825/7497 [00:12<00:11, 308.36it/s]


 51%|█████▏    | 3860/7497 [00:13<00:11, 319.14it/s]


 52%|█████▏    | 3893/7497 [00:13<00:11, 319.71it/s]


 52%|█████▏    | 3928/7497 [00:13<00:10, 327.08it/s]


 53%|█████▎    | 3961/7497 [00:13<00:10, 324.65it/s]


 53%|█████▎    | 3997/7497 [00:13<00:10, 332.40it/s]


 54%|█████▍    | 4031/7497 [00:13<00:10, 332.91it/s]


 54%|█████▍    | 4065/7497 [00:13<00:10, 329.42it/s]


 55%|█████▍    | 4099/7497 [00:13<00:10, 331.87it/s]


 55%|█████▌    | 4133/7497 [00:13<00:10, 333.41it/s]


 56%|█████▌    | 4167/7497 [00:14<00:09, 333.64it/s]


 56%|█████▌    | 4201/7497 [00:14<00:09, 329.99it/s]


 56%|█████▋    | 4235/7497 [00:14<00:10, 325.52it/s]


 57%|█████▋    | 4268/7497 [00:14<00:10, 319.05it/s]


 57%|█████▋    | 4303/7497 [00:14<00:09, 326.50it/s]


 58%|█████▊    | 4336/7497 [00:14<00:09, 322.00it/s]


 58%|█████▊    | 4370/7497 [00:14<00:09, 326.46it/s]


 59%|█████▊    | 4404/7497 [00:14<00:09, 328.75it/s]


 59%|█████

In [12]:
# def risks(risk, page): # risk in [Flooding, Air Pollution, Extreme Heat, Drought, Extreme Weather]
#     p = page.find('div', {'id': 'risks_hazards'})
#     if p is not None: 
#         p = p.find('div', {'class': 'vc_col-sm-6'})
#         for i in p.find_all('p'):
#             if risk in i.text:
#                 if 'default' in i.attrs.get('class', []):
#                     return False
#                 else:
#                     return True

In [13]:
# cities['flood_risk'] = cities['page'].progress_apply(lambda x: risks('Flooding', x))
# cities['air_pollution_risk'] = cities['page'].progress_apply(lambda x: risks('Air Pollution', x))
# cities['extreme_heat_risk'] = cities['page'].progress_apply(lambda x: risks('Extreme Heat', x))
# cities['drought_risk'] = cities['page'].progress_apply(lambda x: risks('Drought', x))
# cities['extreme_weather_risk'] = cities['page'].progress_apply(lambda x: risks('Extreme Weather', x))

In [23]:
def action(action, page): # action in [Transportation, Buildings, Waste, Water, Energy]
    p = page.find('div', {'id': 'risks_hazards'})
    if p is not None:
        p = p.find_all('div', {'class': 'vc_col-sm-6'})[1]
        for i in p.find_all('p'):
            if action in i.text:
                if 'default' in i.attrs.get('class', []):
                    return False
                else:
                    return True

In [24]:
cities['transportation_action'] = cities['page'].progress_apply(lambda x: action('Transportation', x))
cities['buildings_action'] = cities['page'].progress_apply(lambda x: action('Buildings', x))
cities['waste_action'] = cities['page'].progress_apply(lambda x: action('Waste', x))
cities['water_action'] = cities['page'].progress_apply(lambda x: action('Water', x))
cities['energy_action'] = cities['page'].progress_apply(lambda x: action('Energy', x))




  0%|          | 0/7497 [00:00<?, ?it/s]


  0%|          | 33/7497 [00:00<00:23, 324.20it/s]


  1%|          | 68/7497 [00:00<00:22, 329.75it/s]


  1%|▏         | 100/7497 [00:00<00:22, 325.68it/s]


  2%|▏         | 135/7497 [00:00<00:22, 330.98it/s]


  2%|▏         | 171/7497 [00:00<00:21, 337.54it/s]


  3%|▎         | 204/7497 [00:00<00:21, 334.49it/s]


  3%|▎         | 238/7497 [00:00<00:21, 335.55it/s]


  4%|▎         | 273/7497 [00:00<00:21, 337.08it/s]


  4%|▍         | 307/7497 [00:00<00:21, 334.35it/s]


  5%|▍         | 340/7497 [00:01<00:21, 329.32it/s]


  5%|▍         | 374/7497 [00:01<00:21, 330.29it/s]


  5%|▌         | 408/7497 [00:01<00:21, 330.63it/s]


  6%|▌         | 444/7497 [00:01<00:20, 336.66it/s]


  6%|▋         | 478/7497 [00:01<00:21, 331.94it/s]


  7%|▋         | 513/7497 [00:01<00:20, 335.79it/s]


  7%|▋         | 547/7497 [00:01<00:21, 325.70it/s]


  8%|▊         | 580/7497 [00:01<00:21, 318.73it/s]


  8%|▊         | 615/7497 [00:01<00:21

 58%|█████▊    | 4323/7497 [00:17<00:13, 227.29it/s]


 58%|█████▊    | 4347/7497 [00:17<00:13, 227.21it/s]


 58%|█████▊    | 4371/7497 [00:17<00:13, 230.41it/s]


 59%|█████▊    | 4395/7497 [00:17<00:13, 230.70it/s]


 59%|█████▉    | 4419/7497 [00:17<00:15, 204.43it/s]


 59%|█████▉    | 4445/7497 [00:17<00:14, 217.49it/s]


 60%|█████▉    | 4473/7497 [00:17<00:13, 230.93it/s]


 60%|█████▉    | 4497/7497 [00:18<00:12, 231.09it/s]


 60%|██████    | 4527/7497 [00:18<00:12, 247.10it/s]


 61%|██████    | 4558/7497 [00:18<00:11, 262.00it/s]


 61%|██████    | 4586/7497 [00:18<00:10, 265.79it/s]


 62%|██████▏   | 4614/7497 [00:18<00:10, 264.74it/s]


 62%|██████▏   | 4643/7497 [00:18<00:10, 269.91it/s]


 62%|██████▏   | 4671/7497 [00:18<00:10, 269.92it/s]


 63%|██████▎   | 4704/7497 [00:18<00:09, 283.47it/s]


 63%|██████▎   | 4737/7497 [00:18<00:09, 294.11it/s]


 64%|██████▎   | 4768/7497 [00:18<00:09, 295.86it/s]


 64%|██████▍   | 4798/7497 [00:19<00:09, 293.02it/s]


 64%|█████

 17%|█▋        | 1283/7497 [00:04<00:19, 321.78it/s]


 18%|█▊        | 1317/7497 [00:04<00:18, 326.34it/s]


 18%|█▊        | 1351/7497 [00:04<00:18, 328.24it/s]


 19%|█▊        | 1387/7497 [00:04<00:18, 336.47it/s]


 19%|█▉        | 1421/7497 [00:04<00:18, 327.09it/s]


 19%|█▉        | 1454/7497 [00:04<00:21, 284.24it/s]


 20%|█▉        | 1486/7497 [00:04<00:20, 292.66it/s]


 20%|██        | 1517/7497 [00:05<00:20, 292.02it/s]


 21%|██        | 1547/7497 [00:05<00:20, 292.91it/s]


 21%|██        | 1577/7497 [00:05<00:20, 290.08it/s]


 22%|██▏       | 1612/7497 [00:05<00:19, 303.61it/s]


 22%|██▏       | 1646/7497 [00:05<00:18, 311.49it/s]


 22%|██▏       | 1678/7497 [00:05<00:18, 307.87it/s]


 23%|██▎       | 1712/7497 [00:05<00:18, 315.34it/s]


 23%|██▎       | 1744/7497 [00:05<00:18, 316.02it/s]


 24%|██▎       | 1776/7497 [00:05<00:18, 313.25it/s]


 24%|██▍       | 1808/7497 [00:06<00:18, 311.50it/s]


 25%|██▍       | 1840/7497 [00:06<00:18, 311.50it/s]


 25%|██▍  

 84%|████████▍ | 6307/7497 [00:20<00:03, 318.36it/s]


 85%|████████▍ | 6340/7497 [00:20<00:03, 318.29it/s]


 85%|████████▌ | 6377/7497 [00:20<00:03, 330.92it/s]


 86%|████████▌ | 6411/7497 [00:20<00:03, 332.90it/s]


 86%|████████▌ | 6445/7497 [00:20<00:03, 330.39it/s]


 86%|████████▋ | 6479/7497 [00:20<00:03, 327.74it/s]


 87%|████████▋ | 6512/7497 [00:20<00:03, 323.81it/s]


 87%|████████▋ | 6546/7497 [00:20<00:02, 327.81it/s]


 88%|████████▊ | 6582/7497 [00:20<00:02, 334.28it/s]


 88%|████████▊ | 6618/7497 [00:21<00:02, 340.89it/s]


 89%|████████▊ | 6653/7497 [00:21<00:02, 342.89it/s]


 89%|████████▉ | 6688/7497 [00:21<00:02, 333.79it/s]


 90%|████████▉ | 6726/7497 [00:21<00:02, 343.80it/s]


 90%|█████████ | 6762/7497 [00:21<00:02, 345.78it/s]


 91%|█████████ | 6797/7497 [00:21<00:02, 341.28it/s]


 91%|█████████ | 6832/7497 [00:21<00:01, 342.05it/s]


 92%|█████████▏| 6867/7497 [00:21<00:01, 343.66it/s]


 92%|█████████▏| 6902/7497 [00:21<00:01, 339.23it/s]


 93%|█████

 46%|████▌     | 3447/7497 [00:14<00:14, 271.05it/s]


 46%|████▋     | 3475/7497 [00:14<00:15, 266.44it/s]


 47%|████▋     | 3508/7497 [00:14<00:14, 282.28it/s]


 47%|████▋     | 3544/7497 [00:14<00:13, 300.47it/s]


 48%|████▊     | 3577/7497 [00:14<00:12, 306.47it/s]


 48%|████▊     | 3609/7497 [00:15<00:13, 296.03it/s]


 49%|████▊     | 3640/7497 [00:15<00:14, 262.97it/s]


 49%|████▉     | 3670/7497 [00:15<00:14, 270.34it/s]


 49%|████▉     | 3698/7497 [00:15<00:14, 261.17it/s]


 50%|████▉     | 3725/7497 [00:15<00:16, 223.44it/s]


 50%|█████     | 3749/7497 [00:15<00:20, 181.36it/s]


 50%|█████     | 3770/7497 [00:15<00:21, 169.98it/s]


 51%|█████     | 3789/7497 [00:16<00:23, 157.96it/s]


 51%|█████     | 3813/7497 [00:16<00:20, 175.73it/s]


 51%|█████     | 3840/7497 [00:16<00:18, 195.98it/s]


 52%|█████▏    | 3868/7497 [00:16<00:16, 214.02it/s]


 52%|█████▏    | 3892/7497 [00:16<00:17, 205.49it/s]


 52%|█████▏    | 3917/7497 [00:16<00:16, 215.55it/s]


 53%|█████

100%|█████████▉| 7481/7497 [00:31<00:00, 246.98it/s]


100%|██████████| 7497/7497 [00:31<00:00, 234.79it/s]


  0%|          | 0/7497 [00:00<?, ?it/s]


  0%|          | 27/7497 [00:00<00:28, 260.31it/s]


  1%|          | 53/7497 [00:00<00:28, 259.65it/s]


  1%|          | 76/7497 [00:00<00:29, 247.78it/s]


  1%|▏         | 97/7497 [00:00<00:31, 232.96it/s]


  2%|▏         | 116/7497 [00:00<00:34, 213.95it/s]


  2%|▏         | 134/7497 [00:00<00:38, 190.11it/s]


  2%|▏         | 151/7497 [00:00<00:43, 169.50it/s]


  2%|▏         | 167/7497 [00:00<00:53, 138.24it/s]


  2%|▏         | 181/7497 [00:01<01:02, 116.56it/s]


  3%|▎         | 194/7497 [00:01<01:03, 115.11it/s]


  3%|▎         | 206/7497 [00:01<01:13, 98.80it/s] 


  3%|▎         | 225/7497 [00:01<01:03, 115.06it/s]


  3%|▎         | 242/7497 [00:01<00:57, 126.21it/s]


  4%|▎         | 266/7497 [00:01<00:49, 146.94it/s]


  4%|▍         | 283/7497 [00:01<00:52, 138.36it/s]


  4%|▍         | 299/7497 [00:01<00:53, 1

 61%|██████▏   | 4600/7497 [00:16<00:12, 228.85it/s]


 62%|██████▏   | 4625/7497 [00:16<00:12, 233.02it/s]


 62%|██████▏   | 4656/7497 [00:16<00:11, 250.69it/s]


 62%|██████▏   | 4682/7497 [00:16<00:11, 250.69it/s]


 63%|██████▎   | 4711/7497 [00:16<00:10, 258.74it/s]


 63%|██████▎   | 4740/7497 [00:16<00:10, 266.82it/s]


 64%|██████▎   | 4769/7497 [00:16<00:10, 271.31it/s]


 64%|██████▍   | 4797/7497 [00:16<00:10, 263.27it/s]


 64%|██████▍   | 4825/7497 [00:16<00:10, 265.94it/s]


 65%|██████▍   | 4853/7497 [00:16<00:09, 267.72it/s]


 65%|██████▌   | 4884/7497 [00:17<00:09, 277.42it/s]


 66%|██████▌   | 4921/7497 [00:17<00:08, 299.39it/s]


 66%|██████▌   | 4956/7497 [00:17<00:08, 311.95it/s]


 67%|██████▋   | 4991/7497 [00:17<00:07, 320.88it/s]


 67%|██████▋   | 5026/7497 [00:17<00:07, 327.50it/s]


 68%|██████▊   | 5062/7497 [00:17<00:07, 335.94it/s]


 68%|██████▊   | 5096/7497 [00:17<00:07, 324.91it/s]


 68%|██████▊   | 5129/7497 [00:17<00:07, 320.98it/s]


 69%|█████

 15%|█▌        | 1160/7497 [00:05<00:25, 244.27it/s]


 16%|█▌        | 1188/7497 [00:05<00:24, 253.76it/s]


 16%|█▋        | 1223/7497 [00:05<00:22, 275.42it/s]


 17%|█▋        | 1259/7497 [00:05<00:21, 295.77it/s]


 17%|█▋        | 1290/7497 [00:05<00:21, 286.87it/s]


 18%|█▊        | 1320/7497 [00:05<00:21, 286.71it/s]


 18%|█▊        | 1350/7497 [00:05<00:21, 284.21it/s]


 18%|█▊        | 1379/7497 [00:05<00:21, 282.82it/s]


 19%|█▉        | 1408/7497 [00:06<00:21, 279.39it/s]


 19%|█▉        | 1437/7497 [00:06<00:21, 276.72it/s]


 20%|█▉        | 1465/7497 [00:06<00:22, 264.59it/s]


 20%|█▉        | 1493/7497 [00:06<00:22, 267.24it/s]


 20%|██        | 1520/7497 [00:06<00:22, 265.87it/s]


 21%|██        | 1547/7497 [00:06<00:22, 266.09it/s]


 21%|██        | 1580/7497 [00:06<00:20, 281.95it/s]


 22%|██▏       | 1615/7497 [00:06<00:19, 298.88it/s]


 22%|██▏       | 1651/7497 [00:06<00:18, 312.52it/s]


 22%|██▏       | 1685/7497 [00:06<00:18, 319.60it/s]


 23%|██▎  

 72%|███████▏  | 5410/7497 [00:22<00:08, 244.69it/s]


 73%|███████▎  | 5442/7497 [00:22<00:07, 262.14it/s]


 73%|███████▎  | 5474/7497 [00:22<00:07, 276.22it/s]


 74%|███████▎  | 5511/7497 [00:22<00:06, 297.01it/s]


 74%|███████▍  | 5548/7497 [00:22<00:06, 313.85it/s]


 74%|███████▍  | 5583/7497 [00:22<00:05, 323.22it/s]


 75%|███████▍  | 5618/7497 [00:22<00:05, 328.29it/s]


 75%|███████▌  | 5654/7497 [00:23<00:05, 336.50it/s]


 76%|███████▌  | 5691/7497 [00:23<00:05, 342.37it/s]


 76%|███████▋  | 5726/7497 [00:23<00:05, 335.93it/s]


 77%|███████▋  | 5760/7497 [00:23<00:05, 330.54it/s]


 77%|███████▋  | 5795/7497 [00:23<00:05, 333.52it/s]


 78%|███████▊  | 5830/7497 [00:23<00:04, 337.15it/s]


 78%|███████▊  | 5864/7497 [00:23<00:05, 315.18it/s]


 79%|███████▊  | 5899/7497 [00:23<00:04, 324.09it/s]


 79%|███████▉  | 5935/7497 [00:23<00:04, 333.27it/s]


 80%|███████▉  | 5971/7497 [00:23<00:04, 340.16it/s]


 80%|████████  | 6006/7497 [00:24<00:04, 342.33it/s]


 81%|█████

In [25]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497 entries, 0 to 7507
Data columns (total 28 columns):
ID                                 7497 non-null object
URL                                7497 non-null object
country                            7497 non-null object
latitude                           7497 non-null float64
longitude                          7497 non-null float64
name                               7497 non-null object
population                         7497 non-null object
page                               7497 non-null object
commitment_phase                   7497 non-null bool
inventory_phase                    7497 non-null bool
target_phase                       7497 non-null bool
plan_phase                         7497 non-null bool
total_ghg_emissions                186 non-null float64
percent_reduction                  0 non-null object
potential_impact                   0 non-null object
target_year                        0 non-null object
action_plan_

In [26]:
cities_out = cities.drop(['page'], axis=1)

In [27]:
date = dt.datetime.today().strftime("%m.%d.%y")
cities_out.to_csv('../../../../output/actors/Global Covenant/' +date+ 'global_covenant_cities.csv', encoding = "utf-8")